In [1]:
import boto3
from sfn_setup import make_def 
import json
import itertools
import numpy as np
import uuid

In [2]:
# Initialize boto3 handler
rds = boto3.client('rds')

# Describes your currently active databases
DBs = rds.describe_db_instances()['DBInstances']
activeDBs = [ x['DBName'] for x in DBs ]
activeDBs

['finalproject']

In [3]:
DB_NAME = 'finalproject'
INSTANCE_NAME = 'final-instance'

if DB_NAME not in activeDBs:
    # Create a new MySQL database on AWS
    response = rds.create_db_instance(
        DBInstanceIdentifier=INSTANCE_NAME,
        DBName=DB_NAME,
        MasterUsername='username',
        MasterUserPassword='password',
        DBInstanceClass='db.t3.micro',   # This specifies the EC2 instance that will run the datbase
        Engine='MySQL',
        AllocatedStorage=5
    )
rds.get_waiter('db_instance_available').wait(DBInstanceIdentifier=INSTANCE_NAME)

In [4]:
db = rds.describe_db_instances()['DBInstances'][0]
ENDPOINT = db['Endpoint']['Address']
PORT = db['Endpoint']['Port']
DBID = db['DBInstanceIdentifier']

print(DBID,
      "is available at", ENDPOINT,
      "on Port", PORT,
     )

final-instance is available at final-instance.cjasscu8073h.us-east-1.rds.amazonaws.com on Port 3306


In [5]:
SGNAME = db['VpcSecurityGroups'][0]['VpcSecurityGroupId']

# Adjust security permissions
try:
    ec2 = boto3.client('ec2')
    data = ec2.authorize_security_group_ingress(
            GroupId=SGNAME,
            IpPermissions=[
                {'IpProtocol': 'tcp',
                 'FromPort': PORT,
                 'ToPort': PORT,
                 'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
            ]
    )
except ec2.exceptions.ClientError as e:
    if e.response["Error"]["Code"] == 'InvalidPermission.Duplicate':
        print("Permissions already adjusted.")
    else:
        print(e)

Permissions already adjusted.


In [36]:
import mysql.connector

# Connect to the MySQL database
conn =  mysql.connector.connect(host=ENDPOINT, 
                                user="username", 
                                passwd="password", 
                                port=PORT, 
                                database=DB_NAME)
cur = conn.cursor()

In [39]:
# Define the start and end of the range
start = 1
end = 100

# Generate the column definitions
columns = [f"P{i} INT NOT NULL" for i in range(start, end + 1)]

# Join all column definitions into a single string, separated by commas
column_string = ", ".join(columns)

print(column_string)

P1 INT NOT NULL, P2 INT NOT NULL, P3 INT NOT NULL, P4 INT NOT NULL, P5 INT NOT NULL, P6 INT NOT NULL, P7 INT NOT NULL, P8 INT NOT NULL, P9 INT NOT NULL, P10 INT NOT NULL, P11 INT NOT NULL, P12 INT NOT NULL, P13 INT NOT NULL, P14 INT NOT NULL, P15 INT NOT NULL, P16 INT NOT NULL, P17 INT NOT NULL, P18 INT NOT NULL, P19 INT NOT NULL, P20 INT NOT NULL, P21 INT NOT NULL, P22 INT NOT NULL, P23 INT NOT NULL, P24 INT NOT NULL, P25 INT NOT NULL, P26 INT NOT NULL, P27 INT NOT NULL, P28 INT NOT NULL, P29 INT NOT NULL, P30 INT NOT NULL, P31 INT NOT NULL, P32 INT NOT NULL, P33 INT NOT NULL, P34 INT NOT NULL, P35 INT NOT NULL, P36 INT NOT NULL, P37 INT NOT NULL, P38 INT NOT NULL, P39 INT NOT NULL, P40 INT NOT NULL, P41 INT NOT NULL, P42 INT NOT NULL, P43 INT NOT NULL, P44 INT NOT NULL, P45 INT NOT NULL, P46 INT NOT NULL, P47 INT NOT NULL, P48 INT NOT NULL, P49 INT NOT NULL, P50 INT NOT NULL, P51 INT NOT NULL, P52 INT NOT NULL, P53 INT NOT NULL, P54 INT NOT NULL, P55 INT NOT NULL, P56 INT NOT NULL, P

In [42]:
total_lst = []
for j in range(8):
    columns = [f"P{i}T{j} INT NOT NULL" for i in range(start, end + 1)]
    column_string = ", ".join(columns)
    print(column_string)
    total_lst.append(column_string)
total_string = ", ".join(total_lst)

P1T0 INT NOT NULL, P2T0 INT NOT NULL, P3T0 INT NOT NULL, P4T0 INT NOT NULL, P5T0 INT NOT NULL, P6T0 INT NOT NULL, P7T0 INT NOT NULL, P8T0 INT NOT NULL, P9T0 INT NOT NULL, P10T0 INT NOT NULL, P11T0 INT NOT NULL, P12T0 INT NOT NULL, P13T0 INT NOT NULL, P14T0 INT NOT NULL, P15T0 INT NOT NULL, P16T0 INT NOT NULL, P17T0 INT NOT NULL, P18T0 INT NOT NULL, P19T0 INT NOT NULL, P20T0 INT NOT NULL, P21T0 INT NOT NULL, P22T0 INT NOT NULL, P23T0 INT NOT NULL, P24T0 INT NOT NULL, P25T0 INT NOT NULL, P26T0 INT NOT NULL, P27T0 INT NOT NULL, P28T0 INT NOT NULL, P29T0 INT NOT NULL, P30T0 INT NOT NULL, P31T0 INT NOT NULL, P32T0 INT NOT NULL, P33T0 INT NOT NULL, P34T0 INT NOT NULL, P35T0 INT NOT NULL, P36T0 INT NOT NULL, P37T0 INT NOT NULL, P38T0 INT NOT NULL, P39T0 INT NOT NULL, P40T0 INT NOT NULL, P41T0 INT NOT NULL, P42T0 INT NOT NULL, P43T0 INT NOT NULL, P44T0 INT NOT NULL, P45T0 INT NOT NULL, P46T0 INT NOT NULL, P47T0 INT NOT NULL, P48T0 INT NOT NULL, P49T0 INT NOT NULL, P50T0 INT NOT NULL, P51T0 INT

In [41]:
print(f"""
    CREATE TABLE IF NOT EXISTS simulation_data (
        combination_id VARCHAR(36),
        RunId INT NOT NULL,
        iteration INT NOT NULL,
        Step INT NOT NULL,
        num_tasks INT NOT NULL,
        num_nodes INT NOT NULL,
        num_new_edges INT NOT NULL,
        skills_proportion FLOAT NOT NULL,
        prob_memory FLOAT NOT NULL,
        availability FLOAT NOT NULL,
        {column_string},
        {total_string},  
        PRIMARY KEY (combination_id, RunId)""")


    CREATE TABLE IF NOT EXISTS simulation_data (
        combination_id VARCHAR(36),
        RunId INT NOT NULL,
        iteration INT NOT NULL,
        Step INT NOT NULL,
        num_tasks INT NOT NULL,
        num_nodes INT NOT NULL,
        num_new_edges INT NOT NULL,
        skills_proportion FLOAT NOT NULL,
        prob_memory FLOAT NOT NULL,
        availability FLOAT NOT NULL,
        P1 INT NOT NULL, P2 INT NOT NULL, P3 INT NOT NULL, P4 INT NOT NULL, P5 INT NOT NULL, P6 INT NOT NULL, P7 INT NOT NULL, P8 INT NOT NULL, P9 INT NOT NULL, P10 INT NOT NULL, P11 INT NOT NULL, P12 INT NOT NULL, P13 INT NOT NULL, P14 INT NOT NULL, P15 INT NOT NULL, P16 INT NOT NULL, P17 INT NOT NULL, P18 INT NOT NULL, P19 INT NOT NULL, P20 INT NOT NULL, P21 INT NOT NULL, P22 INT NOT NULL, P23 INT NOT NULL, P24 INT NOT NULL, P25 INT NOT NULL, P26 INT NOT NULL, P27 INT NOT NULL, P28 INT NOT NULL, P29 INT NOT NULL, P30 INT NOT NULL, P31 INT NOT NULL, P32 INT NOT NULL, P33 INT NOT NULL, P34 INT NOT NULL, P

In [23]:
cur.execute(f"""
        CREATE TABLE IF NOT EXISTS simulation_data (
        combination_id VARCHAR(36),
        RunId INT NOT NULL,
        iteration INT NOT NULL,
        Step INT NOT NULL,
        num_tasks INT NOT NULL,
        num_nodes INT NOT NULL,
        num_new_edges INT NOT NULL,
        skills_proportion FLOAT NOT NULL,
        prob_memory FLOAT NOT NULL,
        availability FLOAT NOT NULL,
        {column_string},
        time_lst TEXT NOT NULL,
        actor_sequence_lst TEXT NOT NULL,  
        PRIMARY KEY (combination_id, RunId)
    );
"""
)

In [44]:
sql_insert_query = """
INSERT INTO simulation_data 
    (RunId, iteration, Step, num_tasks, num_nodes, num_new_edges, 
    skills_proportion, prob_memory, availability, combination_id, 
    actor_sequence_lst, time_lst) 
VALUES 
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = {'RunId': 0, 'iteration': 0, 'Step': 100, 'num_tasks': 2, 'num_nodes': 10, 'num_new_edges': 1, 'skills_proportion': 0.1, 'prob_memory': 0.0, 'availablity': 0.1, 'combination_id': 'e9afe102-096f-46e5-966a-26dddd6e68bd', 'actor_sequence_lst': '[{0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}, {0: 6, 1: 3}]', 'time_lst': '[75, 38, 38, 41, 51, 47, 35, 40, 47, 37, 44, 65, 30, 52, 53, 39, 76, 28, 49, 43, 52, 23, 69, 41, 61, 39, 57, 30, 36, 33, 51, 51, 36, 48, 25, 37, 87, 55, 37, 24, 47, 59, 28, 29, 40, 42, 35, 55, 51, 39, 35, 39, 30, 35, 61, 32, 46, 28, 40, 37, 31, 57, 41, 44, 59, 34, 34, 37, 73, 37, 37, 18, 44, 37, 27, 33, 46, 19, 34, 19, 122, 36, 28, 48, 41, 40, 29, 55, 33, 41, 32, 43, 40, 38, 57, 31, 27, 32, 18, 31, 62]'}

cur.execute(sql_insert_query, tuple(data.values()))

In [58]:
cur.reset()

In [20]:
cur.execute("DROP TABLE simulation_data;")

In [38]:
cur.execute("SELECT * FROM simulation_data LIMIT 5;")
print(cur.fetchall())

[('03475b04-f841-4f90-aee5-293c831fde38', 1, 0, 100, 8, 20, 2, 0.1, 0.5, 0.5, '[118, 63, 41, 45, 49, 49, 37, 41, 50, 47, 42, 39, 36, 46, 38, 41, 36, 41, 36, 37, 40, 36, 39, 42, 41, 52, 41, 38, 44, 41, 38, 42, 41, 38, 41, 58, 42, 40, 36, 36, 45, 42, 36, 37, 49, 44, 43, 44, 39, 43, 38, 36, 40, 42, 48, 44, 39, 36, 34, 39, 36, 44, 38, 41, 38, 51, 40, 35, 47, 43, 43, 42, 36, 42, 40, 46, 52, 33, 40, 43, 34, 41, 38, 42, 40, 41, 44, 37, 39, 43, 43, 38, 43, 40, 37, 46, 44, 41, 40, 48, 40]', '[{0: 4, 1: 9, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 9, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 9, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 9, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 7, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 7, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 7, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 7, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 7, 2: 6, 3: 10, 4: 15, 5: 15, 6: 11, 7: 11}, {0: 4, 1: 7, 2: 

In [8]:
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
aws_lambda = boto3.client('lambda')

try:
    response = aws_lambda.create_function(
        FunctionName='routine2',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code={
            'S3Bucket': 'abbeynewbucket',
            'S3Key': 'final_project_packages'
        },
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
        S3Bucket='abbeynewbucket',
        S3Key='final_project_packages'
    )

response = aws_lambda.put_function_concurrency(
    FunctionName='routine2',
    ReservedConcurrentExecutions=10
)



InvalidParameterValueException: An error occurred (InvalidParameterValueException) when calling the CreateFunction operation: Unzipped size must be smaller than 262144000 bytes

In [6]:
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
aws_lambda = boto3.client('lambda')

# Open our Zipped directory
with open('my_deployment_package.zip', 'rb') as f:
    lambda_zip = f.read()

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='routine1',
        Runtime='python3.9',
        Role=role['Role']['Arn'],
        Handler='lambda_function.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip
    # file contents
    response = aws_lambda.update_function_code(
        S3Bucket='your-bucket-name',
        S3Key='path/to/your_deployment_package.zip'
    )

response = aws_lambda.put_function_concurrency(
    FunctionName='routine1',
    ReservedConcurrentExecutions=10
)

ClientError: An error occurred (RequestEntityTooLargeException) when calling the CreateFunction operation: Request must be smaller than 70167211 bytes for the CreateFunction operation

In [9]:
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')
aws_lambda = boto3.client('lambda')

lambda_arn = aws_lambda.get_function(FunctionName='routine222')['Configuration']['FunctionArn']
sf_def = make_def(lambda_arn)

In [10]:
sfn = boto3.client('stepfunctions') 

try:
    response = sfn.create_state_machine(
        name='routine_sfn',
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn'],
        type='EXPRESS'
    )
except sfn.exceptions.StateMachineAlreadyExists:
    response = sfn.list_state_machines()
    state_machine_arn = [sm['stateMachineArn'] 
                         for sm in response['stateMachines'] 
                         if sm['name'] == 'routine_sfn'][0]
    response = sfn.update_state_machine(
        stateMachineArn=state_machine_arn,
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn']
    )

# Get arn for Step Function state machine
response = sfn.list_state_machines()
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'routine_sfn'][0]

In [30]:
# Create a unique ID for each combination
from collections.abc import Iterable, Mapping
from typing import Any, Optional, Union

def make_combination_list(parameters: Mapping[str, Union[Any, Iterable[Any]]],
) -> list[dict[str, Any]]:
    """Create model kwargs from parameters dictionary.

    Parameters
    ----------
    parameters : Mapping[str, Union[Any, Iterable[Any]]]
        Single or multiple values for each model parameter name

    Returns
    -------
    List[Dict[str, Any]]
        A list of all kwargs combinations.
    """
    parameter_list = []
    for param, values in parameters.items():
        if isinstance(values, str):
            # The values is a single string, so we shouldn't iterate over it.
            all_values = [(param, values)]
        else:
            try:
                all_values = [(param, value) for value in values]
            except TypeError:
                all_values = [(param, values)]
        parameter_list.append(all_values)

    all_kwargs = itertools.product(*parameter_list)

    kwargs_list = []
    for kwargs in all_kwargs:
        # Create a unique identifier for each combination
        unique_id = str(uuid.uuid4())
        # Convert tuple pairs into a dictionary and add the unique ID
        kwargs_dict = dict(kwargs)
        kwargs_dict['combination_id'] = unique_id
        kwargs_list.append(kwargs_dict)

    return kwargs_list


In [31]:
parameters = {
    "num_tasks": range(3, 20, 3),
    "num_nodes": range(10, 101, 10),
    "num_new_edges": range(1, 9, 1),
    "skills_ratio": np.linspace(0.1, 0.5, 11),
    "prob_memory": np.linspace(0, 1.0, 6),
    "availablity": np.linspace(0.1, 1.0, 6)
}

In [32]:
result = make_combination_list(parameters)

In [33]:
len(result)

190080

In [14]:
result[0]

{'num_tasks': 2,
 'num_nodes': 10,
 'num_new_edges': 1,
 'skills_proportion': 0.1,
 'prob_memory': 0.0,
 'availablity': 0.1,
 'combination_id': '6bfdf22e-8347-47c8-aa44-48dda8c5b68e'}

In [15]:
total_combinations = len(result)
num_workers = 10
chunk_size = total_combinations // num_workers
combination_batches = [result[i:i+chunk_size] for i in range(0, total_combinations, chunk_size)]
# event:{"combinations_lst", iterations, max_steps, data_collection_period}
data = [{"combinations_lst":combination_batches[i],
         "iterations": 50, 
         "max_steps": 100, 
         "data_collection_period": -1} for i in range(num_workers)]

In [16]:
response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name='routine_sfn',
    input=json.dumps(data)
)

ValidationException: An error occurred (ValidationException) when calling the StartSyncExecution operation: 1 validation error detected: Value at 'input' failed to satisfy constraint: Member must have length less than or equal to 262144

In [17]:
cur.execute('SELECT COUNT(*) FROM simulation_data;')
query_results = cur.fetchall()
print(query_results)

[(0,)]
